In [1]:
# Librerias
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import pandas as pd
import numpy as np
from numpy.linalg import norm
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import OneHotEncoder
import re

In [17]:
# importando datos                                                        
with open('annafrank.txt') as f:
    data = f.read()
print(data)

THE DIARY OF A YOUNG GIRL : THE DEFINITIVE EDITION 


Anne Frank 

Edited by Otto H. Frank and Mirjam Pressler 
Translated by Susan Massotty 


BOOK FLAP 

Anne Frank's The Diary of a Young Girl is among the most enduring documents of the 
twentieth century. Since its publication in 1947, it has been read by tens of millions 
of people all over the world. It remains a beloved and deeply admired testament to the 
indestructable nature of the human spirit. 

Restore in this Definitive Edition are diary entries that had been omitted from the 
original edition. These passages, which constitute 30 percent more material, reinforce 
the fact that Anne was first and foremost a teenage girl, not a remote and flawless 
symbol. She fretted about, and tried to copie with, her own emerging sexuality. Like 
many young girls, she often found herself in disagreement with her mother. And like 
any teenager, she veered between the carefree nature of a child and the full-fledged 
sorrow of an adult. Anne

In [3]:
# Descargando recursos semanticos
nltk.download('stopwords')
sw = stopwords.words('english')
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/macbookpro/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
# Preprocesamiento general
data = data.lower() # Conviertiendo texto en minuscula
data = re.sub(r'[,!?;-]', '',data) # Eliminando caracteres raros

In [14]:
# Preprocesamiento por palabras
dataToken = word_tokenize(data)
dataToken = [i for i in dataToken if i.isalpha() or i == '.'] # Eliminando caracter no alfanumericos
dataToken = [i for i in dataToken if i not in sw]
dataToken = [lemmatizer.lemmatize(i) for i in dataToken]
# Sinonimos
dataToken[:20]

['diary',
 'young',
 'girl',
 'definitive',
 'edition',
 'anne',
 'frank',
 'edited',
 'otto',
 'frank',
 'mirjam',
 'pressler',
 'translated',
 'susan',
 'massotty',
 'book',
 'flap',
 'anne',
 'frank',
 'diary']

Posterior al preprocesamiento, se debe crear el vocabulario de las palabras unicas contenidas dentro del texto. Tal como se muestra a continuacion, el texto cuenta con 7087 palabras unicas ordenadas alfabeticamente

In [15]:
# Preprocesamiento por oraciones
dataSent = sent_tokenize(data)
#sentencePre = []
#corpusPro = []
for sentence in dataSent:
    print(word_tokenize(sentence))
#    for word in sentence:
#        print(word)
        #if word.isalpha() and word not in sw:
            #wordLemma = lemmatizer.lemmatize(word)
            #sentencePre.append(wordLemma)
    #corpusPro.append(sentencePre)

['the', 'diary', 'of', 'a', 'young', 'girl', ':', 'the', 'definitive', 'edition', 'anne', 'frank', 'edited', 'by', 'otto', 'h.', 'frank', 'and', 'mirjam', 'pressler', 'translated', 'by', 'susan', 'massotty', 'book', 'flap', 'anne', 'frank', "'s", 'the', 'diary', 'of', 'a', 'young', 'girl', 'is', 'among', 'the', 'most', 'enduring', 'documents', 'of', 'the', 'twentieth', 'century', '.']
['since', 'its', 'publication', 'in', '1947', 'it', 'has', 'been', 'read', 'by', 'tens', 'of', 'millions', 'of', 'people', 'all', 'over', 'the', 'world', '.']
['it', 'remains', 'a', 'beloved', 'and', 'deeply', 'admired', 'testament', 'to', 'the', 'indestructable', 'nature', 'of', 'the', 'human', 'spirit', '.']
['restore', 'in', 'this', 'definitive', 'edition', 'are', 'diary', 'entries', 'that', 'had', 'been', 'omitted', 'from', 'the', 'original', 'edition', '.']
['these', 'passages', 'which', 'constitute', '30', 'percent', 'more', 'material', 'reinforce', 'the', 'fact', 'that', 'anne', 'was', 'first', 'an

In [7]:
dataSent

["the diary of a young girl : the definitive edition \n\n\nanne frank \n\nedited by otto h. frank and mirjam pressler \ntranslated by susan massotty \n\n\nbook flap \n\nanne frank's the diary of a young girl is among the most enduring documents of the \ntwentieth century.",
 'since its publication in 1947. it has been read by tens of millions \nof people all over the world.',
 'it remains a beloved and deeply admired testament to the \nindestructable nature of the human spirit.',
 'restore in this definitive edition are diary entries that had been omitted from the \noriginal edition.',
 'these passages.',
 'which constitute 30 percent more material.',
 'reinforce \nthe fact that anne was first and foremost a teenage girl.',
 'not a remote and flawless \nsymbol.',
 'she fretted about.',
 'and tried to copie with.',
 'her own emerging sexuality.',
 'like \nmany young girls.',
 'she often found herself in disagreement with her mother.',
 'and like \nany teenager.',
 'she veered between th

In [66]:
# Se crea el vocabulario de palabras
voc = set(data)
voc = sorted(voc)
print(len(voc))
print(voc[:20])

7087
['.', 'aachen', 'aagje', 'aah', 'abandon', 'abandoned', 'abduction', 'aber', 'abide', 'ability', 'abject', 'ablaze', 'able', 'abominable', 'abounds', 'aboveboard', 'abruptly', 'absent', 'absentminded', 'absolute']


hablar de :
diferentes metodos de encoding a nivel general (cuales no se van a usar)
datos a utilizar
preprocesamiento


# One Hot Encoder
El One Hot Encoder es una evolucion del indicator encoding. Este ultimo codifica cada palabra en una nueva variable hasta obtener N-1 columnas, siendo N el tamaño del vocabulario. El indicator encoding eliminaba una columna dado que las categorias son mutuamente excluyentes, por lo tanto el tener N variables podria romper el supuesto de independencia entre las palabras que se quieren modelar. Actualmente, esto no es necesario dado que que los modelos modernos generalmente incorporan metodos de regularizacion, disminuyendo la importancia de variables linealmente correlaccionadas.

A partir de lo anterior, surge el one Hot Encoder el cual es uno de los mas usados entre los metodos de conteo, el cual crea un vector para cada palabra y dentro de ese vector se asigna el valor de 1 para representar la presencia de la palabra dentro de una posicion especifica del vocabulario

$$abandono = [0, 0, 0, 0, 1, 0, 0 ..... 0, 0, 0, 0]_{n}$$

Para el ejemplo anterior se muestra la representacion vectorial de la palabra **abandono**, la cual tiene un valor de 1 en la posicion 4 (tomando como posicion inicial el 0). Es decir, dentro del vocabulario, **abandono** se encuentra en la posicion 4. Asimimo, cuando se agregan todos los vectores de las palabras del vocabulario se obtiene una matriz tal y como se muestra a continuacion:

In [30]:
# Aplicando OHE
OHE = OneHotEncoder()
OHE.fit(np.reshape(voc, (-1, 1)))
embeddingOHE = pd.DataFrame(OHE.transform(np.reshape(voc, (-1, 1))).toarray(), columns=OHE.get_feature_names_out())
embeddingOHE.head()

,x0_.,x0_aachen,x0_aagje,x0_aah,x0_abandon,x0_abandoned,x0_abduction,x0_aber,x0_abide,x0_ability,...,x0_zealand,x0_zero,x0_zeus,x0_zhlobin,x0_zionist,x0_zipper,x0_zone,x0_zookeeper,x0_zu,x0_zweite
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Tal como se muestra en la tabla, se crea una matriz simetrica de 7087x7087. El hecho de que sea simetrica indica que el vector que representa cada palabra puede obtenerse al extraer una fila o columna de la matriz. Por ejemplo, para la palabra abandono se puede extraer el vector para ambos componentes y compararlos:

In [59]:
# Extrayendo 10 primeros elementos vector palabra abandon
print("Vector abandono por columnas:", np.array(embeddingOHE['x0_abandon'])[:10])
print("Vector abandono por filas:", np.array(embeddingOHE.iloc[4, :])[:10])

Vector abandono por columnas: [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
Vector abandono por filas: [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]


Esta representacion vectorial de palabras se puede aplicar para representar distintos documentos. En este caso, se tiene un solo texto (Anna Frank), por lo tanto, mas que representar el corpus entero, se podria representar las oraciones dentro del texto al considerarlas cada una como distintos documentos.

Como se presentó anteriormente, una de las principales ventajas del OHE es su facilidad de entendimiento y por ende aplicabilidad a diferentes tipos de problemas tabulares. Sin embargo, una de las principales desventajas es la longitud para representar cada palabra: en el caso anterior se obtienen vectores de longitud 7087, que en otros problemas puede ser mayor.

Otro aspecto negativo relacionado con el anterior es su dependencia al preprocesamiento, es decir, si bien esta etapa es muy importante en cualquier problema de ciencia de datos, este metodo depende al 100% de el dado que si no se hace una correcta eliminacion de stopwords o caracteres extraños, estos haran parte del vocabulario incrementando aun mas la longitud del vector.

Finalmente, la debilidad mas grande del OHE es la poca o nula semantica de los vectores en referencia a sus palabras. Por ejemplo, las palabras **abandon** y **abandoned** son la misma palabra pero en tiempos linguisticos distintos (presente y pasado respectivamente). Aunque se encuentren en tiempos distintos, estas deberian tener cierta relacion dado que linguisticamente representan conceptos similares. Tal similaridad se evalua haciendo uso de la distancia coseno:

$$cosineSimilarity = \frac{A\cdot{B}}{\lVert A \lVert \lVert B \lVert }$$

Donde A y B son los vectores que representan las palabras, en este caso, vectores construidos mediante OHE Encoding. Si la similaridad es 1 significa que estas palabras son iguales, si es -1 indica que son opuestas (antonimos) y si es cero indica que no poseen ninguna clase de relacion.

In [88]:
# Calculando similaridad
cosine_similarity(np.array(embeddingOHE['x0_abandon']).reshape(1, -1),
                  np.array(embeddingOHE['x0_abandoned']).reshape(1, -1))

array([[0.]])

Se observa que para estas dos palabras, la similaridad de su representacion es 0, por lo tanto cualquier algoritmo asumiria que son palabras sin ninguna clase de relacion. De hecho, la similaridad entre dos vectores de palabras counstruidos por este metodo siempre sera cero.